In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import statsmodels.api as sm
import patsy

import time

In [12]:
def preprocess(data):
    tmp = pd.to_datetime(data['Date'])
    data['Wk'] = tmp.dt.isocalendar().week
    data['Yr'] = tmp.dt.year
    data['Wk'] = pd.Categorical(data['Wk'], categories=[i for i in range(1, 53)])  # 52 weeks 
    data['IsHoliday'] = data['IsHoliday'].apply(int)
    return data

In [13]:
train_url = 'train.csv'
test_url = 'test.csv'

In [14]:
    # Reading train data
    file_path = train_url
    train = pd.read_csv(file_path)

    # Reading test data
    file_path = test_url
    test = pd.read_csv(file_path)
    
    #SVD 
    d = 8
    test_depts = test['Dept'].unique()
    train_new = pd.DataFrame()
    for dept in test_depts:
        train_dept_data = train[train['Dept'] == dept]        
        selected_columns = train_dept_data[['Store', 'Date', 'Weekly_Sales']]

        pivoted=selected_columns.pivot(index='Date', columns='Store', values='Weekly_Sales').fillna(0)
        index_names = pivoted.index
        column_names = pivoted.columns
        train_dept_ts = np.array(pivoted)
        
        store_mean = np.mean(train_dept_ts,axis=0)
        train_dept_ts_new = train_dept_ts - store_mean
        U, S, V = np.linalg.svd(train_dept_ts_new)
        d_max = min(d,len(S))
        X_new = np.dot(U[:,:d_max],np.dot(np.diag(S[:d_max]),V[:d_max,:]))+ store_mean
        df = pd.DataFrame(X_new, columns=column_names, index=index_names)
        train_dept_new = pd.melt(df.reset_index(),id_vars='Date', var_name='Store',value_name='Weekly_Sales_SVD')
        train_dept_new['Dept'] = dept
        train_new = pd.concat([train_new, train_dept_new], ignore_index=True)
    train = train.merge(train_new,on = ['Date','Store','Dept'],how='left').fillna(0).drop('Weekly_Sales',axis=1)
    

    # pre-allocate a pd to store the predictions
    test_pred = pd.DataFrame()

    train_pairs = train[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    test_pairs = test[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    unique_pairs = pd.merge(train_pairs, test_pairs, how = 'inner', on =['Store', 'Dept'])

    train_split = unique_pairs.merge(train, on=['Store', 'Dept'], how='left')
    train_split = preprocess(train_split)
    y, X = patsy.dmatrices('Weekly_Sales_SVD ~ Weekly_Sales_SVD + Store + Dept + Yr  + Wk+ IsHoliday', 
                           data = train_split, 
                           return_type='dataframe')
    train_split = dict(tuple(X.groupby(['Store', 'Dept'])))


    test_split = unique_pairs.merge(test, on=['Store', 'Dept'], how='left')
    test_split = preprocess(test_split)
    y, X = patsy.dmatrices('Yr ~ Store + Dept + Yr  + Wk+ IsHoliday', 
                           data = test_split, 
                           return_type='dataframe')
    X['Date'] = test_split['Date']
    test_split = dict(tuple(X.groupby(['Store', 'Dept'])))

    keys = list(train_split)

    for key in keys:
        X_train = train_split[key]
        X_test = test_split[key]
        holidays = X_test['IsHoliday']

        Y = X_train['Weekly_Sales_SVD']
        X_train = X_train.drop(['Weekly_Sales_SVD','Store', 'Dept', 'IsHoliday'], axis=1)
        X_test = X_test.drop(['IsHoliday'], axis=1)
        
        X_train['Yr_square'] = X_train['Yr']**2
        X_test['Yr_square'] = X_test['Yr']**2
        

        cols_to_drop = X_train.columns[(X_train == 0).all()]
        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)

        cols_to_drop = []
        for i in range(len(X_train.columns) - 1, 1, -1):  # Start from the last column and move backward
            col_name = X_train.columns[i]
            # Extract the current column and all previous columns
            tmp_Y = X_train.iloc[:, i].values
            tmp_X = X_train.iloc[:, :i].values

            coefficients, residuals, rank, s = np.linalg.lstsq(tmp_X, tmp_Y, rcond=None)
            if np.sum(residuals) < 1e-16:
                    cols_to_drop.append(col_name)

        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)
        

        model = sm.OLS(Y, X_train).fit()
        mycoef = model.params.fillna(0)
        
        tmp_pred = X_test[['Store', 'Dept', 'Date']]
        X_test = X_test.drop(['Store', 'Dept', 'Date'], axis=1)

        tmp_pred['Weekly_Pred'] = np.dot(X_test, mycoef)
        tmp_pred['IsHoliday'] = holidays.apply(bool)
        test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)

    test_pred['Weekly_Pred'].fillna(0, inplace=True)
    file_path = 'mypred.csv'
    test_pred.to_csv(file_path, index=False)

In [13]:
num_folds = 10
processing_times = []
start = time.time()
for j in range(1, num_folds + 1):
    
    
    # Reading train data
    file_path = f'Proj2_Data/fold_{j}/train.csv'
    train = pd.read_csv(file_path)

    # Reading test data
    file_path = f'Proj2_Data/fold_{j}/test.csv'
    test = pd.read_csv(file_path)
    
    #SVD 
    d = 8
    test_depts = test['Dept'].unique()
    train_new = pd.DataFrame()
    for dept in test_depts:
        train_dept_data = train[train['Dept'] == dept]        
        selected_columns = train_dept_data[['Store', 'Date', 'Weekly_Sales']]

        pivoted=selected_columns.pivot(index='Date', columns='Store', values='Weekly_Sales').fillna(0)
        index_names = pivoted.index
        column_names = pivoted.columns
        train_dept_ts = np.array(pivoted)
        
        store_mean = np.mean(train_dept_ts,axis=0)
        train_dept_ts_new = train_dept_ts - store_mean
        U, S, V = np.linalg.svd(train_dept_ts_new)
        d_max = min(d,len(S))
        X_new = np.dot(U[:,:d_max],np.dot(np.diag(S[:d_max]),V[:d_max,:]))+ store_mean
#        X_new = np.dot(U[:,:len(S)],np.dot(np.diag(S),V[:len(S),:]))+ store_mean
#        if np.sum(X_new-train_dept_ts_new) > 1e-8:
#            print
        df = pd.DataFrame(X_new, columns=column_names, index=index_names)
        train_dept_new = pd.melt(df.reset_index(),id_vars='Date', var_name='Store',value_name='Weekly_Sales_SVD')
        train_dept_new['Dept'] = dept
        train_new = pd.concat([train_new, train_dept_new], ignore_index=True)
    train = train.merge(train_new,on = ['Date','Store','Dept'],how='left').fillna(0).drop('Weekly_Sales',axis=1)
    

    # pre-allocate a pd to store the predictions
    test_pred = pd.DataFrame()

    train_pairs = train[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    test_pairs = test[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    unique_pairs = pd.merge(train_pairs, test_pairs, how = 'inner', on =['Store', 'Dept'])

    train_split = unique_pairs.merge(train, on=['Store', 'Dept'], how='left')
    train_split = preprocess(train_split)
    y, X = patsy.dmatrices('Weekly_Sales_SVD ~ Weekly_Sales_SVD + Store + Dept + Yr  + Wk+ IsHoliday', 
                           data = train_split, 
                           return_type='dataframe')
    train_split = dict(tuple(X.groupby(['Store', 'Dept'])))


    test_split = unique_pairs.merge(test, on=['Store', 'Dept'], how='left')
    test_split = preprocess(test_split)
    y, X = patsy.dmatrices('Yr ~ Store + Dept + Yr  + Wk+ IsHoliday', 
                           data = test_split, 
                           return_type='dataframe')
    X['Date'] = test_split['Date']
    test_split = dict(tuple(X.groupby(['Store', 'Dept'])))

    keys = list(train_split)

    for key in keys:
        X_train = train_split[key]
        X_test = test_split[key]
        holidays = X_test['IsHoliday']

        Y = X_train['Weekly_Sales_SVD']
#        weights = 1/np.sqrt(X_train['IsHoliday']*4 + 1)
        X_train = X_train.drop(['Weekly_Sales_SVD','Store', 'Dept', 'IsHoliday'], axis=1)
        X_test = X_test.drop(['IsHoliday'], axis=1)
        
        X_train['Yr_square'] = X_train['Yr']**2
        X_test['Yr_square'] = X_test['Yr']**2
        

        cols_to_drop = X_train.columns[(X_train == 0).all()]
        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)

        cols_to_drop = []
        for i in range(len(X_train.columns) - 1, 1, -1):  # Start from the last column and move backward
            col_name = X_train.columns[i]
            # Extract the current column and all previous columns
            tmp_Y = X_train.iloc[:, i].values
            tmp_X = X_train.iloc[:, :i].values

            coefficients, residuals, rank, s = np.linalg.lstsq(tmp_X, tmp_Y, rcond=None)
            if np.sum(residuals) < 1e-16:
                    cols_to_drop.append(col_name)

        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)
        

        model = sm.OLS(Y, X_train).fit()
#        model = sm.WLS(Y, X_train,weights).fit()
        mycoef = model.params.fillna(0)

        tmp_pred = X_test[['Store', 'Dept', 'Date']]
        X_test = X_test.drop(['Store', 'Dept', 'Date'], axis=1)

        tmp_pred['Weekly_Pred'] = np.dot(X_test, mycoef)
        tmp_pred['IsHoliday'] = holidays
        test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)

    test_pred['Weekly_Pred'].fillna(0, inplace=True)
    file_path = f'Proj2_Data/fold_{j}/mypred.csv'
    test_pred.to_csv(file_path, index=False)
    
    end = time.time()
    processing_times.append(round(end-start,2))
    
    print('fold%i is processed'%j)
    print('time eclapsed: %0.2f seconds'%(end - start))
    start = end

fold1 is processed
time eclapsed: 25.48 seconds
fold2 is processed
time eclapsed: 27.48 seconds
fold3 is processed
time eclapsed: 33.62 seconds
fold4 is processed
time eclapsed: 34.62 seconds
fold5 is processed
time eclapsed: 39.62 seconds
fold6 is processed
time eclapsed: 38.81 seconds
fold7 is processed
time eclapsed: 43.69 seconds
fold8 is processed
time eclapsed: 48.75 seconds
fold9 is processed
time eclapsed: 43.49 seconds
fold10 is processed
time eclapsed: 43.70 seconds


In [15]:
def myeval():
    file_path = 'Proj2_Data/test_with_label.csv'
    test_with_label = pd.read_csv(file_path)
    num_folds = 10
    wae = []

    for i in range(num_folds):
        file_path = f'Proj2_Data/fold_{i+1}/test.csv'
        test = pd.read_csv(file_path)
        test = test.drop(columns=['IsHoliday']).merge(test_with_label, on=['Date', 'Store', 'Dept'])

        file_path = f'Proj2_Data/fold_{i+1}/mypred.csv'
        test_pred = pd.read_csv(file_path)

        # Left join with the test data
        new_test = test_pred.merge(test, on=['Date', 'Store', 'Dept'], how='left')

        # Compute the Weighted Absolute Error
        actuals = new_test['Weekly_Sales']
        preds = new_test['Weekly_Pred']
        weights = new_test['IsHoliday_x'].apply(lambda x: 5 if x else 1)
        wae.append(sum(weights * abs(actuals - preds)) / sum(weights))

    return wae

In [16]:
wae = myeval()
for value in wae:
    print(f"\t{value:.3f}")
print(f"{sum(wae) / len(wae):.3f}")

	1947.662
	1391.496
	1393.792
	1524.559
	2318.163
	1637.733
	1615.955
	1362.799
	1351.377
	1332.592
1587.613


In [17]:
[round(ele,1) for ele in wae]

[1947.7,
 1391.5,
 1393.8,
 1524.6,
 2318.2,
 1637.7,
 1616.0,
 1362.8,
 1351.4,
 1332.6]

In [18]:
processing_times

[25.48, 27.48, 33.62, 34.62, 39.62, 38.81, 43.69, 48.75, 43.49, 43.7]